In [0]:
%sql
-- Creating the external location
CREATE EXTERNAL LOCATION IF NOT EXISTS external_data_landing_zone
URL 'abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone'
WITH (CREDENTIAL `yogesh-storage-credential`);


In [0]:
%fs ls abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone

path,name,size,modificationTime
abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone/customers/,customers/,0,1708849950000
abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone/customers_1.csv,customers_1.csv,412,1708847263000
abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone/invoices/,invoices/,0,1708849970000
abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone/invoices_2022.csv,invoices_2022.csv,22449,1708847263000
abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone/random/,random/,0,1708851191000


In [0]:
%sql
-- Creating the dev catalog and it's internal objects
CREATE CATALOG IF NOT EXISTS dev;
CREATE DATABASE IF NOT EXISTS dev.demo_db;
CREATE DATABASE IF NOT EXISTS dev.silver_db;
CREATE DATABASE IF NOT EXISTS dev.bronze_db;
CREATE DATABASE IF NOT EXISTS dev.gold_db;

In [0]:
%sql
-- Creating the Volume pointing to our Data Landing Zone

CREATE EXTERNAL VOLUME IF NOT EXISTS dev.demo_db.landing_zone
LOCATION 'abfss://landing-zone@yogeshstoragee.dfs.core.windows.net/data_landing_zone';

In [0]:
%sql
-- Creating the customer_cleaned table
CREATE TABLE IF NOT EXISTS dev.silver_db.customer_cleaned
(
  customer_id int,
  customer_name STRING,
  load_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
)
PARTITIONED BY (load_time)
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql
-- Creating the invoice_cleaned table
CREATE TABLE IF NOT EXISTS dev.silver_db.invoice_cleaned
(
  invoice_no INT,
  stock_code STRING,
  description STRING,
  quantity INT,
  invoice_date DATE,
  unit_price DOUBLE,
  customer_id INT,
  country STRING,
  load_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
)
PARTITIONED BY (load_time)
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql
-- Creating the customers table 
CREATE TABLE IF NOT EXISTS dev.silver_db.customers
(
  customer_id int,
  customer_name STRING,
  start_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
  end_timestamp TIMESTAMP DEFAULT NULL,
  isRecordLatest BOOLEAN DEFAULT TRUE
) 
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql
-- creating the invoices table
CREATE TABLE IF NOT EXISTS dev.silver_db.invoices
PARTITIONED BY (load_time)
AS
SELECT invoice_no, stock_code, description, quantity, invoice_date, unit_price, customer_id, country, current_timestamp() as load_time
FROM dev.silver_db.invoice_cleaned
WHERE date(load_time)=date(current_timestamp())
group by invoice_no, stock_code, description, quantity, invoice_date, unit_price, customer_id, country

In [0]:
%fs ls /Volumes/dev/demo_db/landing_zone/

path,name,size,modificationTime
dbfs:/Volumes/dev/demo_db/landing_zone/customers/,customers/,0,1708849950000
dbfs:/Volumes/dev/demo_db/landing_zone/customers_1.csv,customers_1.csv,412,1708847263000
dbfs:/Volumes/dev/demo_db/landing_zone/invoices/,invoices/,0,1708849970000
dbfs:/Volumes/dev/demo_db/landing_zone/invoices_2022.csv,invoices_2022.csv,22449,1708847263000
dbfs:/Volumes/dev/demo_db/landing_zone/random/,random/,0,1708851191000


In [0]:

%fs mkdirs /Volumes/dev/demo_db/landing_zone/customers/

res2: Boolean = true

In [0]:
%fs mkdirs /Volumes/dev/demo_db/landing_zone/invoices/

res3: Boolean = true

In [0]:
%sql
select current_timestamp()

current_timestamp()
2024-02-27T03:59:20.771Z


In [0]:
%sql
-- Some Testing work
create table if not exists dev.demo_db.temp
(
  id int,
  name string,
  load_time timestamp default current_timestamp()
)
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql
insert into dev.demo_db.temp(id, name)
values
(3, 'Tiwari'),
(4, 'Goel')

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select *
from dev.demo_db.temp

id,name,load_time
1,Yogesh,2024-02-27T04:54:30.68Z
2,Sachin,2024-02-27T04:54:30.68Z


In [0]:
%sql
describe history dev.demo_db.temp

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-02-27T04:58:21Z,3332869799178646,yogeshtiwari389@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(866266771837229),0226-151405-d7c1repa,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 1102)",null,Databricks-Runtime/13.3.x-scala2.12
1,2024-02-27T04:54:34Z,3332869799178646,yogeshtiwari389@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(866266771837229),0226-151405-d7c1repa,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 1108)",null,Databricks-Runtime/13.3.x-scala2.12
0,2024-02-27T04:53:44Z,3332869799178646,yogeshtiwari389@gmail.com,CREATE TABLE,"Map(partitionBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(866266771837229),0226-151405-d7c1repa,null,WriteSerializable,true,Map(),null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
%sql
select * from dev.demo_db.temp version as of 1

id,name,load_time
1,Yogesh,2024-02-27T04:54:30.68Z
2,Sachin,2024-02-27T04:54:30.68Z


In [0]:
%sql
restore table dev.demo_db.temp to version as of 1

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1108,1,1,0,1102,0
